## **Gaining Google Drive Authentication**

In [0]:
from google.colab import auth
auth.authenticate_user()

## **Creating Required Directories**




In [0]:
mkdir result
mkdir Train
mkdir Test

### **Checking directories**

In [77]:
cd

/content


## **Code to upload image file into google colab**

In [0]:
from google.colab import files
uploaded = files.upload()

for name, data in uploaded.items():
  with open(name, 'wb') as f:
    f.write(data)
    print ('saved file', name)

## **Clear directory**

In [0]:
!rm -rf Train
!mkdir Train

## **Checking all directories**

In [0]:
ls

## **Download result images**

In [0]:
from google.colab import files
files.download('greyscale1.png')

## **Display Result Image**

In [0]:
from matplotlib import pyplot as plt
img = plt.imread('result/img_4.png')
plt.imshow(img)

# ** Main Code**

## **Importing Libraries**

In [0]:
#Import Libraries

from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf
# Getting images
X = []
for filename in os.listdir('Train'):
    X.append(img_to_array(load_img('Train/'+filename)))
X = np.array(X,dtype=float)

## **Training Data**

In [71]:

# Set up training and test data

split = int(0.95*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain
#Design the neural network
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
# Finish model
model.compile(optimizer='rmsprop', loss='mse')
# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)
# Generate training data
batch_size = 50
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)
# Train model
TensorBoard(log_dir='/output')

model.fit_generator(image_a_b_gen(batch_size), steps_per_epoch=2000, epochs=1)
#Saving the neural network model
model.save('model2000.h5')

    

Epoch 1/1
2000/2000 [==============================] - 5505s 3s/step - loss: 0.0164


## **Download the neural network model**

In [0]:
  from google.colab import files
  files.download('model2000.h5')

## **Testing Images**

In [0]:
# Test images
from keras.models import load_model

model = load_model('model2000.h5')

Xtest = rgb2lab(1.0/255*X[split:])[:,:,:,0]
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = rgb2lab(1.0/255*X[split:])[:,:,:,1:]
Ytest = Ytest / 128
print (model.evaluate(Xtest, Ytest, batch_size=batch_size))
# Load black and white images
color_me = []
for filename in os.listdir('Test/'):
        color_me.append(img_to_array(load_img('Test/'+filename)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))
# Test model
output = model.predict(color_me)
output = output * 128
# Output colorizations
for i in range(len(output)):
        cur = np.zeros((256, 256, 3))
        cur[:,:,0] = color_me[i][:,:,0]
        cur[:,:,1:] = output[i]
        imsave("result/img_"+str(i)+".png", lab2rgb(cur))
        
model.save('model.h5')